In [ ]:
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal, XSD
from rdflib.namespace import RDF, OWL, FOAF,RDFS
import numpy as np

In [ ]:
g = Graph()

# create namespce for process resource and ontology
CLBR = Namespace("https://climatebowl.data.dice-research.org/resource/")
g.bind("clbr",CLBR)

CLBO = Namespace("https://climatebowl.data.dice-research.org/ontology/")
g.bind("clbo", CLBO)

ontology_file = "ontology.ttl"
g.parse(ontology_file, format="turtle")

In [ ]:

def process_excel_to_rdf(pcf_sheet, Process_worksheet_name, company, product):
    # Define the namespace
    CLBO = Namespace("https://climatebowl.data.dice-research.org/ontology/")
    # Read the Excel sheet
    df = pd.read_excel(pcf_sheet, header=3)
    
    # Fill NaN values with an empty string
    df = df.fillna("")
    print(df)
    
    
    # Initialize the graph
    g = Graph()
    
    # Create URIs for the company and product
    companyURI = URIRef(f"https://climatebowl.data.dice-research.org/resource/company_{company}")
    g.add((companyURI, RDF.type, CLBO.Company))
    g.add((companyURI, RDFS.label, Literal(company)))

    productURI = URIRef(f"https://climatebowl.data.dice-research.org/resource/product_{company}_{product}")
    g.add((productURI, RDF.type, CLBO.Product))
    g.add((productURI, RDFS.label, Literal(product)))
    g.add((companyURI, CLBO.hatProduct, productURI))

    # Process each row in the dataframe
    prozess_no = 1
    fluzz_input = 1
    emission_no = 1
    material_no = 1

    for index, row in df.iterrows():
        # Create URIs for the process and fluzz resource
        prozess_uri = URIRef(f"https://climatebowl.data.dice-research.org/resource/Prozess_{company}_{product}_{prozess_no}")
        fluzz_resouce_uri = URIRef(f"https://climatebowl.data.dice-research.org/resource/{company}_{product}_InputProzess_{prozess_no - 1}Fluzz{fluzz_input}")
        hat_io_ontology_uri = URIRef(f"https://climatebowl.data.dice-research.org/ontology/hatInput")
        
        g.add((prozess_uri, RDF.type, CLBO.Prozess))
        g.add((productURI, CLBO.hatProzess, prozess_uri))
        g.add((prozess_uri, CLBO.prozessmodul, Literal(row["Prozessmodul"])))
        g.add((prozess_uri, CLBO.zuordnungTeilenummer, Literal(row["Zuordnung Komponente"])))
        g.add((prozess_uri, hat_io_ontology_uri, fluzz_resouce_uri))
        
        # Process the fluzz resource
        g.add((fluzz_resouce_uri, RDF.type, CLBO.Fluss))
        relevantPCF = row["relevant für PCF"] == "x"
        g.add((fluzz_resouce_uri, CLBO.relevantPCF, Literal(relevantPCF, datatype=XSD.boolean)))
        
        if row["Zuordnung Wertschöpfungskette"]:
            g.add((fluzz_resouce_uri, CLBO.zuordnungWertschöpfungskette, Literal(row["Zuordnung Wertschöpfungskette"])))
        if row["Scope"]:
            g.add((fluzz_resouce_uri, CLBO.scope, Literal(row["Scope"])))
        if row["Lebenszyklusphase"]:
            g.add((fluzz_resouce_uri, CLBO.lebenszyklusphase, Literal(row["Lebenszyklusphase"])))
        
        # Process kategorie fluss
        kategoryFluss = str(row['Kategorie Fluss']).replace(" ","")
        if kategoryFluss:
            kategorie_fluss_uri = URIRef(f"https://climatebowl.data.dice-research.org/resource/{company}_{product}_{kategoryFluss}{material_no}")
            g.add((fluzz_resouce_uri, CLBO.hatKategorie, kategorie_fluss_uri))
            g.add((kategorie_fluss_uri, RDF.type, getattr(CLBO, kategoryFluss)))
            g.add((kategorie_fluss_uri, RDFS.label, Literal(row["Fluss"])))
            material_no += 1
        
        # Add resultierende Menge
        if row["Menge"]:
            g.add((fluzz_resouce_uri, URIRef(f"https://climatebowl.data.dice-research.org/ontology/resultierendeMengejeJeReferenzfluss"), Literal(float(row["Menge"]), datatype=XSD.float)))
        if row["Einheit"]:
            g.add((fluzz_resouce_uri, URIRef(f"https://climatebowl.data.dice-research.org/ontology/einheitResultierendeMengejeJeReferenzfluss"), Literal(row["Einheit"])))
        
        # Add emission factor
        emissionFactorURI = URIRef(f"https://climatebowl.data.dice-research.org/resource/THGEmissionen{emission_no}_{company}_{product}")
        g.add((fluzz_resouce_uri, CLBO.hatEmissionsfaktor, emissionFactorURI))
        g.add((emissionFactorURI, RDF.type, CLBO.THGEmissionen))
        g.add((emissionFactorURI, CLBO.item, Literal(row["Item Emissionsfaktor"])))
        g.add((emissionFactorURI, CLBO.emissionsfaktor, Literal(float(row["Emissionsfaktor (GWP100 pro Einheit nach IPCC AR6)"]), datatype=XSD.float)))
        g.add((emissionFactorURI, CLBO.einheit, Literal(row["Einheit Emissionsfaktor"])))
        g.add((emissionFactorURI, CLBO.land, Literal(row["Region"])))
        g.add((emissionFactorURI, CLBO.jahr, Literal(row["Jahr"])))
        g.add((emissionFactorURI, CLBO.datenquelle, Literal(row["Datenquelle"])))
        emission_no += 1
        
        # Add material
        materialURI = URIRef(f"https://climatebowl.data.dice-research.org/resource/material{material_no}_{company}_{product}")
        g.add((materialURI, RDF.type, getattr(CLBO, str(row["Gruppe Fluss"]).replace(", ", "_"))))
        g.add((materialURI, CLBO.itemName, Literal(row["Fluss"])))
        g.add((materialURI, CLBO.emissionenWert, Literal(float(row["Emissionsfaktor (GWP100 pro Einheit nach IPCC AR6)"]), datatype=XSD.float)))
        g.add((materialURI, CLBO.emissionenEinheit, Literal(row["Einheit Emissionsfaktor"])))
        g.add((materialURI, CLBO.region, Literal(row["Region"])))
        g.add((materialURI, CLBO.jahr, Literal(row["Jahr"])))
        material_no += 1
        
        # Increment process and fluzz input counters
        prozess_no += 1
        fluzz_input += 1
    
    return g

# Example usage
# g = process_excel_to_rdf('path_to_file.xlsx', 'Sheet1', 'company1', 'product1')


In [ ]:
import json
# Read the JSON data from the file
with open("data.json", 'r') as file:
    data = json.load(file)

# Loop through each item in the JSON data
for item in data:
    product_name = item['productName']
    company_name = item['companyName']
    file_name = item['fileName']
    sheet_name = item['sheetName']
    
    # Call the function to process the Excel sheet and add to the RDF graph
    g += process_excel_to_rdf(f"excel_data/{file_name}", sheet_name, company_name, product_name)



In [ ]:

# Export the graph to a Turtle file
with open('generatedRDG/knowledgeGraph.ttl', 'w', encoding='utf8') as f:
    f.write(g.serialize(format='turtle'))